## Classification
---
Let's try to train support vector machine classification model to predict whether flights will be delayed, cancelled. or on time.


### Load Data File

In [1]:
%matplotlib inline
import math
import matplotlib.pyplot as plt
from decimal import *
import numpy as np # linear algebra
import pandas as pd # read_csv and such
from io import StringIO # convert strings to buffers or something like that.
import os # for listing files in directory
import seaborn as sns
import scipy.stats as stats

pd.options.display.max_columns = 99
pd.options.display.max_rows = 99

In [2]:
flights_weather_path = '../data/flight_delays_2015/flights_weather.csv'
dtypes = {
    'ORIGIN_AIRPORT': 'str', 
    'DESTINATION_AIRPORT': 'str', 
    'IATA_CODE_x': 'str', 
    'origin_weather_station': 'str', 
    'IATA_CODE_y': 'str', 
    'destination_weather_station': 'str', 
    'OR_MAX': 'str', 
    'OR_MIN': 'str', 
    'OR_PRCP': 'str', 
    'DES_MAX': 'str', 
    'DES_MIN': 'str', 
    'DES_PRCP': 'str', 
    'OR_FRSHTT': 'str', 
    'DES_FRSHTT': 'str'
}
fw_df = pd.read_csv(flights_weather_path, dtype=dtypes, parse_dates=['DATE'])
# fw_df = fw_df[['DATE', 'ORIGIN_AIRPORT', 'DESTINATION_AIRPORT', 'WEATHER_DELAY', 'OR_TEMP', 'OR_VISIB',
#                'OR_WDSP', 'OR_MXSPD', 'OR_SNDP', 'OR_PRCP', 'OR_GUST', 'OR_MAX', 'OR_MIN', 'OR_FOG',
#                'OR_RAIN_DRIZZLE', 'OR_SNOW_ICE_PELLETS', 'OR_HAIL', 'OR_THUNDER', 'OR_TORNADO_FUNNEL_CLOUD',
#                'DES_TEMP', 'DES_VISIB', 'DES_WDSP', 'DES_MXSPD', 'DES_SNDP', 'DES_PRCP', 'DES_GUST', 'DES_MAX',
#                'DES_MIN', 'DES_FOG', 'DES_RAIN_DRIZZLE', 'DES_SNOW_ICE_PELLETS', 'DES_HAIL', 'DES_THUNDER',
#                'DES_TORNADO_FUNNEL_CLOUD']]
fw_df = fw_df.sample(n=300000)
fw_df.head()

,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,DEPARTURE_TIME,DEPARTURE_DELAY,TAXI_OUT,WHEELS_OFF,SCHEDULED_TIME,ELAPSED_TIME,AIR_TIME,DISTANCE,WHEELS_ON,TAXI_IN,SCHEDULED_ARRIVAL,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY,DATE,IATA_CODE_x,origin_weather_station,IATA_CODE_y,destination_weather_station,OR_TEMP,OR_COUNT_TEMP,OR_DEWP,OR_COUNT_DEWP,OR_SLP,OR_COUNT_SLP,OR_STP,OR_COUNT_STP,OR_VISIB,OR_COUNT_VISIB,OR_WDSP,OR_COUNT_WDSP,OR_MXSPD,OR_GUST,OR_MAX,OR_MIN,OR_PRCP,OR_SNDP,OR_FRSHTT,DES_TEMP,DES_COUNT_TEMP,DES_DEWP,DES_COUNT_DEWP,DES_SLP,DES_COUNT_SLP,DES_STP,DES_COUNT_STP,DES_VISIB,DES_COUNT_VISIB,DES_WDSP,DES_COUNT_WDSP,DES_MXSPD,DES_GUST,DES_MAX,DES_MIN,DES_PRCP,DES_SNDP,DES_FRSHTT,OR_FOG,OR_RAIN_DRIZZLE,OR_SNOW_ICE_PELLETS,OR_HAIL,OR_THUNDER,OR_TORNADO_FUNNEL_CLOUD,DES_FOG,DES_RAIN_DRIZZLE,DES_SNOW_ICE_PELLETS,DES_HAIL,DES_THUNDER,DES_TORNADO_FUNNEL_CLOUD
2344407,2015,8,11,2,EV,6178,N16571,IAD,EWR,1230,1232.0,2.0,21.0,1253.0,86.0,74.0,47.0,212,1340.0,6.0,1356,1346.0,-10.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,2015-08-11,IAD,724030-93738,EWR,725020-14734,77.5,24,68.2,24,1009.9,21,998.6,24,9.8,24,7.7,24,12.0,17.1,89.1,69.1,0.03G,999.9,010000,75.1,24,67.3,24,1010.8,21,1009.6,24,9.1,24,8.9,24,20.0,26.0,82.4*,71.1*,0.66G,999.9,010000,False,True,False,False,False,False,False,True,False,False,False,False
4608878,2015,5,19,2,DL,2525,N958AT,ATL,BHM,1755,1804.0,9.0,14.0,1818.0,58.0,44.0,28.0,134,1746.0,2.0,1753,1748.0,-5.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,2015-05-19,ATL,722190-13874,BHM,722280-13876,74.8,24,66.6,24,1018.5,24,982.2,24,10.0,24,4.5,24,8.0,999.9,84.9,69.1,1.19G,999.9,010000,75.7,24,67.4,24,1019.3,24,997.2,24,10.0,24,2.9,24,8.0,999.9,86.0,70.0,0.00G,999.9,010000,False,True,False,False,False,False,False,True,False,False,False,False
2725546,2015,8,17,1,AA,1898,N555AY,CLT,JFK,2209,2211.0,2.0,15.0,2226.0,110.0,109.0,83.0,541,2349.0,11.0,2359,2400.0,1.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,2015-08-17,CLT,723140-13881,JFK,744860-94789,81.8,24,64.5,24,1019.7,24,992.8,24,10.0,24,3.7,24,8.9,15.0,93.0,66.9,0.00G,999.9,000000,80.5,24,68.5,24,1018.5,24,1017.7,24,10.0,24,7.4,24,15.9,21.0,91.0,73.9,0.00G,999.9,000000,False,False,False,False,False,False,False,False,False,False,False,False
1529606,2015,5,28,4,EV,4604,N14188,RIC,IAH,1758,1756.0,-2.0,16.0,1812.0,199.0,186.0,162.0,1157,1954.0,8.0,2017,2002.0,-15.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,2015-05-28,RIC,724010-13740,IAH,722430-12960,78.8,24,67.2,24,1022.2,19,1015.8,24,10.0,24,6.8,24,13.0,17.1,91.0,70.0,0.00G,999.9,010010,79.2,24,72.3,24,1017.2,22,1013.5,24,10.0,24,5.6,24,13.0,19.0,90.0,64.9,0.59G,999.9,000000,False,True,False,False,True,False,False,False,False,False,False,False
2336398,2015,7,23,4,AA,1518,N3GFAA,DFW,EWR,2020,2055.0,35.0,12.0,2107.0,217.0,191.0,173.0,1372,100.0,6.0,57,106.0,9.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,2015-07-23,DFW,722590-03927,EWR,725020-14734,89.6,24,67.6,24,1011.7,24,991.2,24,10.0,24,10.5,24,14.0,18.1,99.0,80.1,0.00G,999.9,000000,77.3,24,49.7,24,1012.3,24,1011.3,24,10.0,24,7.6,24,15.0,26.0,87.1,66.9,0.00G,999.9,000000,False,False,False,False,False,False,False,False,False,False,False,False


In [3]:
fw_df['OR_PRCP'] = pd.to_numeric(fw_df['OR_PRCP'].str.replace('A|B|C|D|E|F|G|H|I', ''))
fw_df['OR_MAX'] = pd.to_numeric(fw_df['OR_MAX'].str.replace('*', ''))
fw_df['OR_MIN'] = pd.to_numeric(fw_df['OR_MIN'].str.replace('*', ''))
fw_df.loc[fw_df['OR_TEMP'] == 9999.9, 'OR_TEMP'] = fw_df.OR_TEMP.mean()
fw_df.loc[fw_df['OR_WDSP'] == 999.9, 'OR_WDSP'] = fw_df.OR_WDSP.mean()
fw_df.loc[fw_df['OR_PRCP'] == 99.99, 'OR_PRCP'] = 0
fw_df.loc[fw_df['OR_VISIB'] == 999.9, 'OR_VISIB'] = fw_df.OR_VISIB.mean()
fw_df.loc[fw_df['OR_GUST'] == 999.9, 'OR_GUST'] = fw_df.OR_GUST.mean()
fw_df.loc[fw_df['OR_MAX'] == 9999.9, 'OR_MAX'] = fw_df.OR_MAX.mean()
fw_df.loc[fw_df['OR_MIN'] == 9999.9, 'OR_MIN'] = fw_df.OR_MIN.mean()
fw_df.loc[fw_df['OR_WDSP'] == 999.9, 'OR_WDSP'] = fw_df.OR_WDSP.mean()
fw_df.loc[fw_df['OR_MXSPD'] == 999.9, 'OR_MXSPD'] = fw_df.OR_MXSPD.mean()
fw_df.loc[fw_df['OR_SNDP'] == 999.9, 'OR_SNDP'] = 0
fw_df.loc[fw_df['OR_FOG'] == True, 'OR_FOGV'] = 1
fw_df.loc[fw_df['OR_FOG'] == False, 'OR_FOGV'] = 0
fw_df.loc[fw_df['OR_RAIN_DRIZZLE'] == True, 'OR_RAIN_DRIZZLEV'] = 1
fw_df.loc[fw_df['OR_RAIN_DRIZZLE'] == False, 'OR_RAIN_DRIZZLEV'] = 0
fw_df.loc[fw_df['OR_SNOW_ICE_PELLETS'] == True, 'OR_SNOW_ICE_PELLETSV'] = 1
fw_df.loc[fw_df['OR_SNOW_ICE_PELLETS'] == False, 'OR_SNOW_ICE_PELLETSV'] = 0
fw_df.loc[fw_df['OR_HAIL'] == True, 'OR_HAILV'] = 1
fw_df.loc[fw_df['OR_HAIL'] == False, 'OR_HAILV'] = 0
fw_df.loc[fw_df['OR_THUNDER'] == True, 'OR_THUNDERV'] = 1
fw_df.loc[fw_df['OR_THUNDER'] == False, 'OR_THUNDERV'] = 0
fw_df.loc[fw_df['OR_TORNADO_FUNNEL_CLOUD'] == True, 'OR_TORNADO_FUNNEL_CLOUDV'] = 1
fw_df.loc[fw_df['OR_TORNADO_FUNNEL_CLOUD'] == False, 'OR_TORNADO_FUNNEL_CLOUDV'] = 0
fw_df['DES_PRCP'] = pd.to_numeric(fw_df['DES_PRCP'].str.replace('A|B|C|D|E|F|G|H|I', ''))
fw_df['DES_MAX'] = pd.to_numeric(fw_df['DES_MAX'].str.replace('*', ''))
fw_df['DES_MIN'] = pd.to_numeric(fw_df['DES_MIN'].str.replace('*', ''))
fw_df.loc[fw_df['DES_TEMP'] == 9999.9, 'DES_TEMP'] = fw_df.DES_TEMP.mean()
fw_df.loc[fw_df['DES_WDSP'] == 999.9, 'DES_WDSP'] = fw_df.DES_WDSP.mean()
fw_df.loc[fw_df['DES_PRCP'] == 99.99, 'DES_PRCP'] = 0
fw_df.loc[fw_df['DES_VISIB'] == 999.9, 'DES_VISIB'] = fw_df.DES_VISIB.mean()
fw_df.loc[fw_df['DES_GUST'] == 999.9, 'DES_GUST'] = fw_df.DES_GUST.mean()
fw_df.loc[fw_df['DES_MAX'] == 9999.9, 'DES_MAX'] = fw_df.DES_MAX.mean()
fw_df.loc[fw_df['DES_MIN'] == 9999.9, 'DES_MIN'] = fw_df.DES_MIN.mean()
fw_df.loc[fw_df['DES_WDSP'] == 999.9, 'DES_WDSP'] = fw_df.DES_WDSP.mean()
fw_df.loc[fw_df['DES_MXSPD'] == 999.9, 'DES_MXSPD'] = fw_df.DES_MXSPD.mean()
fw_df.loc[fw_df['DES_SNDP'] == 999.9, 'DES_SNDP'] = 0
fw_df.loc[fw_df['DES_FOG'] == True, 'DES_FOGV'] = 1
fw_df.loc[fw_df['DES_FOG'] == False, 'DES_FOGV'] = 0
fw_df.loc[fw_df['DES_RAIN_DRIZZLE'] == True, 'DES_RAIN_DRIZZLEV'] = 1
fw_df.loc[fw_df['DES_RAIN_DRIZZLE'] == False, 'DES_RAIN_DRIZZLEV'] = 0
fw_df.loc[fw_df['DES_SNOW_ICE_PELLETS'] == True, 'DES_SNOW_ICE_PELLETSV'] = 1
fw_df.loc[fw_df['DES_SNOW_ICE_PELLETS'] == False, 'DES_SNOW_ICE_PELLETSV'] = 0
fw_df.loc[fw_df['DES_HAIL'] == True, 'DES_HAILV'] = 1
fw_df.loc[fw_df['DES_HAIL'] == False, 'DES_HAILV'] = 0
fw_df.loc[fw_df['DES_THUNDER'] == True, 'DES_THUNDERV'] = 1
fw_df.loc[fw_df['DES_THUNDER'] == False, 'DES_THUNDERV'] = 0
fw_df.loc[fw_df['DES_TORNADO_FUNNEL_CLOUD'] == True, 'DES_TORNADO_FUNNEL_CLOUDV'] = 1
fw_df.loc[fw_df['DES_TORNADO_FUNNEL_CLOUD'] == False, 'DES_TORNADO_FUNNEL_CLOUDV'] = 0
fw_df.loc[fw_df['WEATHER_DELAY'].isnull(), 'WEATHER_DELAY'] = 0


Remove flights that are cancelled for any reason other than weather.

In [4]:
fw_df = fw_df[fw_df['CANCELLATION_REASON'] != 'A']
fw_df = fw_df[fw_df['CANCELLATION_REASON'] != 'C']
fw_df = fw_df[fw_df['CANCELLATION_REASON'] != 'D']
fw_df.head()

,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,DEPARTURE_TIME,DEPARTURE_DELAY,TAXI_OUT,WHEELS_OFF,SCHEDULED_TIME,ELAPSED_TIME,AIR_TIME,DISTANCE,WHEELS_ON,TAXI_IN,SCHEDULED_ARRIVAL,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY,DATE,IATA_CODE_x,origin_weather_station,IATA_CODE_y,destination_weather_station,OR_TEMP,OR_COUNT_TEMP,OR_DEWP,OR_COUNT_DEWP,OR_SLP,OR_COUNT_SLP,OR_STP,OR_COUNT_STP,OR_VISIB,OR_COUNT_VISIB,OR_WDSP,OR_COUNT_WDSP,OR_MXSPD,OR_GUST,OR_MAX,OR_MIN,OR_PRCP,OR_SNDP,OR_FRSHTT,DES_TEMP,DES_COUNT_TEMP,DES_DEWP,DES_COUNT_DEWP,DES_SLP,DES_COUNT_SLP,DES_STP,DES_COUNT_STP,DES_VISIB,DES_COUNT_VISIB,DES_WDSP,DES_COUNT_WDSP,DES_MXSPD,DES_GUST,DES_MAX,DES_MIN,DES_PRCP,DES_SNDP,DES_FRSHTT,OR_FOG,OR_RAIN_DRIZZLE,OR_SNOW_ICE_PELLETS,OR_HAIL,OR_THUNDER,OR_TORNADO_FUNNEL_CLOUD,DES_FOG,DES_RAIN_DRIZZLE,DES_SNOW_ICE_PELLETS,DES_HAIL,DES_THUNDER,DES_TORNADO_FUNNEL_CLOUD,OR_FOGV,OR_RAIN_DRIZZLEV,OR_SNOW_ICE_PELLETSV,OR_HAILV,OR_THUNDERV,OR_TORNADO_FUNNEL_CLOUDV,DES_FOGV,DES_RAIN_DRIZZLEV,DES_SNOW_ICE_PELLETSV,DES_HAILV,DES_THUNDERV,DES_TORNADO_FUNNEL_CLOUDV
2344407,2015,8,11,2,EV,6178,N16571,IAD,EWR,1230,1232.0,2.0,21.0,1253.0,86.0,74.0,47.0,212,1340.0,6.0,1356,1346.0,-10.0,0,0,NaN,NaN,NaN,NaN,NaN,0.0,2015-08-11,IAD,724030-93738,EWR,725020-14734,77.5,24,68.2,24,1009.9,21,998.6,24,9.8,24,7.7,24,12.0,17.100000,89.1,69.1,0.03,0.0,010000,75.1,24,67.3,24,1010.8,21,1009.6,24,9.1,24,8.9,24,20.0,26.000000,82.4,71.1,0.66,0.0,010000,False,True,False,False,False,False,False,True,False,False,False,False,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4608878,2015,5,19,2,DL,2525,N958AT,ATL,BHM,1755,1804.0,9.0,14.0,1818.0,58.0,44.0,28.0,134,1746.0,2.0,1753,1748.0,-5.0,0,0,NaN,NaN,NaN,NaN,NaN,0.0,2015-05-19,ATL,722190-13874,BHM,722280-13876,74.8,24,66.6,24,1018.5,24,982.2,24,10.0,24,4.5,24,8.0,427.294027,84.9,69.1,1.19,0.0,010000,75.7,24,67.4,24,1019.3,24,997.2,24,10.0,24,2.9,24,8.0,425.203433,86.0,70.0,0.00,0.0,010000,False,True,False,False,False,False,False,True,False,False,False,False,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2725546,2015,8,17,1,AA,1898,N555AY,CLT,JFK,2209,2211.0,2.0,15.0,2226.0,110.0,109.0,83.0,541,2349.0,11.0,2359,2400.0,1.0,0,0,NaN,NaN,NaN,NaN,NaN,0.0,2015-08-17,CLT,723140-13881,JFK,744860-94789,81.8,24,64.5,24,1019.7,24,992.8,24,10.0,24,3.7,24,8.9,15.000000,93.0,66.9,0.00,0.0,000000,80.5,24,68.5,24,1018.5,24,1017.7,24,10.0,24,7.4,24,15.9,21.000000,91.0,73.9,0.00,0.0,000000,False,False,False,False,False,False,False,False,False,False,False,False,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1529606,2015,5,28,4,EV,4604,N14188,RIC,IAH,1758,1756.0,-2.0,16.0,1812.0,199.0,186.0,162.0,1157,1954.0,8.0,2017,2002.0,-15.0,0,0,NaN,NaN,NaN,NaN,NaN,0.0,2015-05-28,RIC,724010-13740,IAH,722430-12960,78.8,24,67.2,24,1022.2,19,1015.8,24,10.0,24,6.8,24,13.0,17.100000,91.0,70.0,0.00,0.0,010010,79.2,24,72.3,24,1017.2,22,1013.5,24,10.0,24,5.6,24,13.0,19.000000,90.0,64.9,0.59,0.0,000000,False,True,False,False,True,False,False,False,False,False,False,False,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2336398,2015,7,23,4,AA,1518,N3GFAA,DFW,EWR,2020,2055.0,35.0,12.0,2107.0,217.0,191.0,173.0,1372,100.0,6.0,57,106.0,9.0,0,0,NaN,NaN,NaN,NaN,NaN,0.0,2015-07-23,DFW,722590-03927,EWR,725020-14734,89.6,24,67.6,24,1011.7,24,991.2,24,10.0,24,10.5,24,14.0,18.100000,99.0,80.1,0.00,0.0,000000,77.3,24,49.7,24,1012.3,24,1011.3,24,10.0,24,7.6,24,15.0,26.000000,87.1,66.9,0.00,0.0,000000,False,False,False,False,False,False,False,False,False,False,False,False,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Create a classifier column for whether the flight is unaffected by weather (1), delayed by weather (2), cancelled due to weather (3).

In [36]:
fw_df['WEATHER_EFFECT'] = 1
fw_df.loc[fw_df['CANCELLATION_REASON'] == 'B', 'WEATHER_EFFECT'] = 3
fw_df.loc[fw_df['WEATHER_DELAY'] > 2, 'WEATHER_EFFECT'] = 2

In [37]:
fw_df.head()

,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,DEPARTURE_TIME,DEPARTURE_DELAY,TAXI_OUT,WHEELS_OFF,SCHEDULED_TIME,ELAPSED_TIME,AIR_TIME,DISTANCE,WHEELS_ON,TAXI_IN,SCHEDULED_ARRIVAL,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY,DATE,IATA_CODE_x,origin_weather_station,IATA_CODE_y,destination_weather_station,OR_TEMP,OR_COUNT_TEMP,OR_DEWP,OR_COUNT_DEWP,OR_SLP,OR_COUNT_SLP,OR_STP,OR_COUNT_STP,OR_VISIB,OR_COUNT_VISIB,OR_WDSP,OR_COUNT_WDSP,OR_MXSPD,OR_GUST,OR_MAX,OR_MIN,OR_PRCP,OR_SNDP,OR_FRSHTT,DES_TEMP,DES_COUNT_TEMP,DES_DEWP,DES_COUNT_DEWP,DES_SLP,DES_COUNT_SLP,DES_STP,DES_COUNT_STP,DES_VISIB,DES_COUNT_VISIB,DES_WDSP,DES_COUNT_WDSP,DES_MXSPD,DES_GUST,DES_MAX,DES_MIN,DES_PRCP,DES_SNDP,DES_FRSHTT,OR_FOG,OR_RAIN_DRIZZLE,OR_SNOW_ICE_PELLETS,OR_HAIL,OR_THUNDER,OR_TORNADO_FUNNEL_CLOUD,DES_FOG,DES_RAIN_DRIZZLE,DES_SNOW_ICE_PELLETS,DES_HAIL,DES_THUNDER,DES_TORNADO_FUNNEL_CLOUD,OR_FOGV,OR_RAIN_DRIZZLEV,OR_SNOW_ICE_PELLETSV,OR_HAILV,OR_THUNDERV,OR_TORNADO_FUNNEL_CLOUDV,DES_FOGV,DES_RAIN_DRIZZLEV,DES_SNOW_ICE_PELLETSV,DES_HAILV,DES_THUNDERV,DES_TORNADO_FUNNEL_CLOUDV,WEATHER_EFFECT
2344407,2015,8,11,2,EV,6178,N16571,IAD,EWR,1230,1232.0,2.0,21.0,1253.0,86.0,74.0,47.0,212,1340.0,6.0,1356,1346.0,-10.0,0,0,NaN,NaN,NaN,NaN,NaN,0.0,2015-08-11,IAD,724030-93738,EWR,725020-14734,77.5,24,68.2,24,1009.9,21,998.6,24,9.8,24,7.7,24,12.0,17.100000,89.1,69.1,0.03,0.0,010000,75.1,24,67.3,24,1010.8,21,1009.6,24,9.1,24,8.9,24,20.0,26.000000,82.4,71.1,0.66,0.0,010000,False,True,False,False,False,False,False,True,False,False,False,False,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1
4608878,2015,5,19,2,DL,2525,N958AT,ATL,BHM,1755,1804.0,9.0,14.0,1818.0,58.0,44.0,28.0,134,1746.0,2.0,1753,1748.0,-5.0,0,0,NaN,NaN,NaN,NaN,NaN,0.0,2015-05-19,ATL,722190-13874,BHM,722280-13876,74.8,24,66.6,24,1018.5,24,982.2,24,10.0,24,4.5,24,8.0,427.294027,84.9,69.1,1.19,0.0,010000,75.7,24,67.4,24,1019.3,24,997.2,24,10.0,24,2.9,24,8.0,425.203433,86.0,70.0,0.00,0.0,010000,False,True,False,False,False,False,False,True,False,False,False,False,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1
2725546,2015,8,17,1,AA,1898,N555AY,CLT,JFK,2209,2211.0,2.0,15.0,2226.0,110.0,109.0,83.0,541,2349.0,11.0,2359,2400.0,1.0,0,0,NaN,NaN,NaN,NaN,NaN,0.0,2015-08-17,CLT,723140-13881,JFK,744860-94789,81.8,24,64.5,24,1019.7,24,992.8,24,10.0,24,3.7,24,8.9,15.000000,93.0,66.9,0.00,0.0,000000,80.5,24,68.5,24,1018.5,24,1017.7,24,10.0,24,7.4,24,15.9,21.000000,91.0,73.9,0.00,0.0,000000,False,False,False,False,False,False,False,False,False,False,False,False,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
1529606,2015,5,28,4,EV,4604,N14188,RIC,IAH,1758,1756.0,-2.0,16.0,1812.0,199.0,186.0,162.0,1157,1954.0,8.0,2017,2002.0,-15.0,0,0,NaN,NaN,NaN,NaN,NaN,0.0,2015-05-28,RIC,724010-13740,IAH,722430-12960,78.8,24,67.2,24,1022.2,19,1015.8,24,10.0,24,6.8,24,13.0,17.100000,91.0,70.0,0.00,0.0,010010,79.2,24,72.3,24,1017.2,22,1013.5,24,10.0,24,5.6,24,13.0,19.000000,90.0,64.9,0.59,0.0,000000,False,True,False,False,True,False,False,False,False,False,False,False,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
2336398,2015,7,23,4,AA,1518,N3GFAA,DFW,EWR,2020,2055.0,35.0,12.0,2107.0,217.0,191.0,173.0,1372,100.0,6.0,57,106.0,9.0,0,0,NaN,NaN,NaN,NaN,NaN,0.0,2015-07-23,DFW,722590-03927,EWR,725020-14734,89.6,24,67.6,24,1011.7,24,991.2,24,10.0,24,10.5,24,14.0,18.100000,99.0,80.1,0.00,0.0,000000,77.3,24,49.7,24,1012.3,24,1011.3,24,10.0,24,7.6,24,15.0,26.000000,87.1,66.9,0.00,0.0,000000,False,False,False,False,False,False,False,False,False,False,False,False,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1


In [40]:
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(fw_df[['OR_TEMP', 'OR_VISIB',
               'OR_WDSP', 'OR_MXSPD', 'OR_SNDP', 'OR_PRCP','OR_MAX', 'OR_MIN', 'OR_FOGV',
               'OR_RAIN_DRIZZLEV', 'OR_SNOW_ICE_PELLETSV', 'OR_HAILV', 'OR_THUNDERV', 'OR_TORNADO_FUNNEL_CLOUDV',
               'DES_TEMP', 'DES_VISIB', 'DES_WDSP', 'DES_MXSPD', 'DES_SNDP', 'DES_PRCP','DES_MAX',
               'DES_MIN', 'DES_FOGV', 'DES_RAIN_DRIZZLEV', 'DES_SNOW_ICE_PELLETSV', 'DES_HAILV', 'DES_THUNDERV',
               'DES_TORNADO_FUNNEL_CLOUDV']], fw_df[['WEATHER_EFFECT']])

#### SVM with rbf kernel

In [41]:
from sklearn.svm import SVC # "Support Vector Classifier"
clf = SVC(kernel='rbf')
clf.fit(X_train, y_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

Now that we have trained the model, we can test it's accuracy.

In [42]:
clf.score(X_test, y_test)

0.97991536239672195

In [43]:
import datetime
datetime.datetime.now()

datetime.datetime(2017, 11, 28, 3, 47, 50, 706573)

At first glance an accuracy of 98% seems good but let's check a confusion matrix for a sample of the test data.

In [44]:
predicted = clf.predict(X_test)

In [45]:
from sklearn.metrics import confusion_matrix
print 'confusion matrix\n', confusion_matrix(y_test, predicted)

confusion matrix
[[72892    19    21]
 [  824    12     6]
 [  622     3    36]]


As suspected the confusion matrix indicates that the model mostly just predicts no effect from weather. Due to how skewed the data is, it may be hard to train on the full data set. Lets remove the no effect and see if it can classify between delays and cancelled flights.


In [47]:
fw_delayed = fw_df[fw_df['WEATHER_EFFECT'] != 1]

In [52]:
X_train, X_test, y_train, y_test = train_test_split(fw_delayed[['OR_TEMP', 'OR_VISIB',
               'OR_WDSP', 'OR_MXSPD', 'OR_SNDP', 'OR_PRCP','OR_MAX', 'OR_MIN', 'OR_FOGV',
               'OR_RAIN_DRIZZLEV', 'OR_SNOW_ICE_PELLETSV', 'OR_HAILV', 'OR_THUNDERV', 'OR_TORNADO_FUNNEL_CLOUDV',
               'DES_TEMP', 'DES_VISIB', 'DES_WDSP', 'DES_MXSPD', 'DES_SNDP', 'DES_PRCP','DES_MAX',
               'DES_MIN', 'DES_FOGV', 'DES_RAIN_DRIZZLEV', 'DES_SNOW_ICE_PELLETSV', 'DES_HAILV', 'DES_THUNDERV',
               'DES_TORNADO_FUNNEL_CLOUDV']], fw_delayed[['WEATHER_EFFECT']])

In [53]:
clf = SVC(kernel='rbf')
clf.fit(X_train, y_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [54]:
clf.score(X_test, y_test)

0.62665782493368705

In [55]:
predicted = clf.predict(X_test)
print 'confusion matrix\n', confusion_matrix(y_test, predicted)

confusion matrix
[[762  66]
 [497 183]]


Without zeros we get an accuracy of about 63%. This is a little better than the previous model since the data is split 55/45 delayed/cancelled. Looking at the confusion matrix though, the model is classifying too many cancelled flights as delayed.